# Test 1 - Site explicit modelling under Mott-Schottky conditions.

**Running the code in this notebook takes approximately 2 minutes. (iMac with 4 Ghz i7 processor).**

This notebook can be run to verify that the code is working as expected using the site explicit model under Mott-Schottky conditions, without including the additional site charge.
The data collected by running this notebook is a DataFrame, saved in `output_data.csv` which contains information about the conditions applied to the calculation, the $x$ coordinate data, and the calculated electrostatic potential, charge density and input and output defect mole fraction. A DataFrame stored in `key_data.dat`, which includes the calculated space charge properties and a saved figure for the elctrostatic potential, charge density and defect mole fractions as a function of $x$.
The outputs from running this notebook will be stored in the `generated_outputs` directory and can be compared to the controlled outputs previously calculated in `expected_outputs` directory. 

In [1]:
from pyscses.defect_species import DefectSpecies
from pyscses.set_of_sites import SetOfSites
from pyscses.constants import boltzmann_eV
from pyscses.calculation import Calculation, calculate_activation_energies
from pyscses.set_up_calculation import calculate_grid_offsets
from pyscses.grid import Grid

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
if not os.path.isdir('generated_outputs'):
    os.mkdir('generated_outputs')

In [3]:
boundary_conditions = 'periodic'
site_charges = False
systems = 'mott-schottky'
core_models = False
site_models = 'site_explicit'
scaling = True

In [4]:
alpha = 0.0005
conv = 1e-6
grid_x_min = -6.094e-9
grid_x_max = 5.16e-9
bulk_x_min = -5.783e-9
bulk_x_max = -2.502e-9

dielectric = 55

index = 111
b = 7.65327e-10
c = 7.65327e-10

temp = [ 773.15 ]

In [5]:
valence = [ +2.0, -1.0 ]
site_labels = [ 'O', 'Ce' ]
defect_labels = ['Vo', 'Gd']
mole_fractions = np.array([ [ 0.05, 0.2 ] ])
initial_guess = np.array([ [ 0.05, 0.2 ] ])
mobilities = [ 1.0, 0.0 ]

In [6]:
data = '../input_data/Gd_CeO2_111_data.txt'

In [7]:
labels=[ 'core_model', 'site_model', 'MS_GC', 'site_charge', 'resistivity_scaling', 'temperature', 'input_mole_fractions', 'x', 'phi', 'rho', 'Vo_mole_fraction', 'Gd_mole_fraction']
output_data = pd.DataFrame( columns=labels )
filename = 'generated_outputs/output_data.csv'

In [8]:
limits, laplacian_limits = calculate_grid_offsets(data,
                                                  grid_x_min,
                                                  grid_x_max,
                                                  'single')







600
[-3.12466845e-08 -3.11685737e-08 -3.10904628e-08 -3.09342411e-08
 -3.08561303e-08 -3.07780194e-08 -3.06217978e-08 -3.05436869e-08
 -3.04655761e-08 -3.03093544e-08 -3.02312435e-08 -3.01531327e-08
 -2.99969110e-08 -2.99188001e-08 -2.98406893e-08 -2.96844676e-08
 -2.96063567e-08 -2.95282459e-08 -2.93720242e-08 -2.92939133e-08
 -2.92158025e-08 -2.90595808e-08 -2.89814700e-08 -2.89033591e-08
 -2.87471374e-08 -2.86690266e-08 -2.85909157e-08 -2.84346940e-08
 -2.83565832e-08 -2.82784723e-08 -2.81222506e-08 -2.80441398e-08
 -2.79660289e-08 -2.78098072e-08 -2.77316964e-08 -2.76535855e-08
 -2.74973638e-08 -2.74192530e-08 -2.73411421e-08 -2.71849205e-08
 -2.71068096e-08 -2.70286988e-08 -2.68724770e-08 -2.67943662e-08
 -2.67162554e-08 -2.65600337e-08 -2.64819228e-08 -2.64038120e-08
 -2.62475903e-08 -2.61694794e-08 -2.60913686e-08 -2.59351469e-08
 -2.58570360e-08 -2.57789252e-08 -2.56227035e-08 -2.55445926e-08
 -2.54664818e-08 -2.53102601e-08 -2.52321493e-08 -2.51540384e-08
 -2.49978167e-08 -2.4

In [9]:
from pyscses.set_up_calculation import sites_data_from_file
from pyscses.set_up_calculation import calculate_grid_offsets_from_sites_data

sites_data = sites_data_from_file(data, x_limits=(grid_x_min, grid_x_max))

print(len(sites_data))
calculate_grid_offsets_from_sites_data(sites_data,
                                       x_limits=(grid_x_min, grid_x_max))

2402
2402
432
432
432
[-5.93876866e-09 -5.86065753e-09 -5.78254641e-09 -5.62632445e-09
 -5.54821333e-09 -5.47010220e-09 -5.31388024e-09 -5.23576912e-09
 -5.15765800e-09 -5.00143603e-09 -4.92332491e-09 -4.84521379e-09
 -4.68899182e-09 -4.61088070e-09 -4.53276958e-09 -4.37654762e-09
 -4.29843650e-09 -4.22032537e-09 -4.06410341e-09 -3.98599229e-09
 -3.90788117e-09 -3.75165920e-09 -3.67354808e-09 -3.59543696e-09
 -3.43921499e-09 -3.36110387e-09 -3.28299275e-09 -3.12677079e-09
 -3.04865966e-09 -2.97054854e-09 -2.81432658e-09 -2.73621546e-09
 -2.65810434e-09 -2.50188237e-09 -2.42377125e-09 -2.34566013e-09
 -2.18943816e-09 -2.11132704e-09 -2.03321592e-09 -1.87699396e-09
 -1.79888283e-09 -1.72077171e-09 -1.56454975e-09 -1.48643863e-09
 -1.40832750e-09 -1.25210555e-09 -1.17399442e-09 -1.09588335e-09
 -9.39661290e-10 -8.61550356e-10 -7.83436652e-10 -6.27222961e-10
 -5.49104275e-10 -4.71146572e-10 -3.14474082e-10 -2.37176149e-10
 -1.60540875e-10 -1.73473580e-12  7.69762842e-11  1.56942899e-10
  3

In [ ]:
for m in mole_fractions:
    for t in temp:
    
        defect_species = {l: DefectSpecies(label=l,
                                           valence=v,
                                           mole_fraction=m,
                                           mobility=mob)
                          for l, v, m, mob
                          in zip(defect_labels, valence, m, mobilities)}

        all_sites = SetOfSites.from_input_data(filename=data,
                                               limits=[grid_x_min, grid_x_max],
                                               defect_species=defect_species,
                                               site_charge=site_charges,
                                               core=core_models,
                                               temperature=t)
        if site_models == 'continuum':
            all_sites, limits = SetOfSites.form_continuum_sites( all_sites, grid_x_min, grid_x_max, 1000, b, c, defect_species, laplacian_limits, site_labels, defect_labels )
        if systems == 'mott-schottky':
            for site in all_sites.subset( 'Ce' ):
                site.defect_with_label('Gd').fixed = True
        if systems == 'gouy-chapman':
            for site in all_sites.subset( 'Ce' ):
                site.defect_with_label('Gd').fixed = False
        grid = Grid.from_set_of_sites( all_sites, limits, laplacian_limits, b, c )
        
        c_o = Calculation( grid, bulk_x_min, bulk_x_max, alpha, conv, dielectric, t, boundary_conditions )
        c_o.form_subgrids( site_labels )
        if systems == 'gouy-chapman':
            c_o.mole_fraction_correction( m, systems, initial_guess )
        c_o.solve(systems, verbose=True)
        c_o.mole_fractions()
        c_o.calculate_resistivity_ratio( 'positive', 2e-2 )
        c_o.solve_MS_approx_for_phi( valence[0] )
        
        output_data['x'] = grid.x
        output_data['phi'] = c_o.phi
        output_data['rho'] = c_o.rho
        output_data['Vo_mole_fraction'] = c_o.mf[site_labels[0]]
        output_data['Gd_mole_fraction'] = c_o.mf[site_labels[1]]
        output_data['core_model'] = str(core_models)
        output_data['site_model'] = str(site_models)
        output_data['MS_GC'] = str(systems)
        output_data['site_charge'] = str(site_charges)
        output_data['resistivity_scaling'] = str(scaling)
        output_data['temperature'] = t
        output_data['input_mole_fractions'] = str(m)
        output_data.to_csv(filename, index=False)
        
        key_data = np.column_stack(( max(c_o.phi), c_o.perpendicular_resistivity_ratio, c_o.parallel_resistivity_ratio ))
        np.savetxt('generated_outputs/key_data.dat', key_data)

In [ ]:
# from fig_params import rcParams
# import fig_params as fp
# rcParams.update( fp.transitionfig_formatting)
plt.plot(grid.x, c_o.phi)
plt.xlabel( '$x$ $\mathrm{coordinate}$' )
plt.ylabel('$\Phi$ $\mathrm{( eV )}$')
plt.xlim(-5e-9, 5e-9)
plt.ylim(min(c_o.phi))
plt.savefig('generated_outputs/x_vs_potential.pdf')
plt.show()

plt.plot(grid.x, c_o.rho)
plt.xlabel( '$x$ $\mathrm{coordinate}$'  )
plt.ylabel(' $ \\rho $ $(\mathrm{C m}^{-1})$')
plt.xlim( -5e-9, 5e-9)
plt.ylim(min(c_o.rho))
plt.savefig('generated_outputs/x_vs_chargedensity.pdf')
plt.show()

plt.plot(grid.x, c_o.mf[site_labels[0]], label = '$\mathrm{Vo}^{..}$', zorder = 5.0)
plt.plot(grid.x, c_o.mf[site_labels[1]], label = '$\mathrm{Gd}$', zorder = 4.0)
plt.xlabel( '$x$ $\mathrm{coordinate}$'  )
plt.ylabel('$x_{i}$')
plt.xlim( -5e-9, 5e-9)
plt.ylim(0.0)
plt.legend()
plt.savefig('generated_outputs/x_vs_molefractions.pdf')
plt.show()